In [1]:
import os
print(os.getcwd())

In [1]:
import torch
from pytorch_lightning.callbacks import LearningRateMonitor
from pytorch_lightning.loggers import WandbLogger
from torch.utils.data import DataLoader
from absl import app
from absl import flags
from pytorch_lightning.utilities.model_summary import ModelSummary


import sys,os
sys.path.append(os.path.dirname(os.path.abspath(os.path.dirname(os.path.abspath(os.path.dirname(os.getcwd()))))))
from datasets.weather_bench import WeatherDataset
from models.VariableAnalyzer.datasets.dataset import CustomDataset
from models.VariableAnalyzer.models.model import VariableAnalyzer
from models.VariableAnalyzer.training.configs import TrainingRunConfig
from models.VariableAnalyzer.training.lightning import TrainModule
from models.VariableAnalyzer.training.callbacks import VariableVaildationCallback

In [2]:
def get_dataset(year_offset: int, src_time_len: int, tgt_time_len: int):
    device = ("cuda" if torch.cuda.is_available() else "cpu" )
    device = torch.device(device)

    weather = WeatherDataset(year_offset, device=device)
    # dataset.shape:  torch.Size([7309, 100, 1450])
    input, target, mean_std, dims = weather.load()
    dataset = CustomDataset(input, target, src_time_len, tgt_time_len)
    return (weather.HAS_LEVEL_VARIABLE, weather.NONE_LEVEL_VARIABLE, weather.PRESSURE_LEVELS), dataset, input.shape, mean_std, dims

src_time_len = 4 * 1
tgt_time_len = 4 * 3

# shape = (time, var, hidden)
dataset_info, dataset, shape, mean_std, dims = get_dataset(0, src_time_len, tgt_time_len)

데이터셋 불러오는 중...
<xarray.DataArray 'level' (level: 13)>
array([  50,  100,  150,  200,  250,  300,  400,  500,  600,  700,  850,  925,
       1000])
Coordinates:
  * level    (level) int64 50 100 150 200 250 300 400 500 600 700 850 925 1000
<xarray.DataArray 'level' (level: 13)>
array([  50,  100,  150,  200,  250,  300,  400,  500,  600,  700,  850,  925,
       1000])
Coordinates:
  * level    (level) int64 50 100 150 200 250 300 400 500 600 700 850 925 1000
<xarray.DataArray 'level' (level: 13)>
array([  50,  100,  150,  200,  250,  300,  400,  500,  600,  700,  850,  925,
       1000])
Coordinates:
  * level    (level) int64 50 100 150 200 250 300 400 500 600 700 850 925 1000
==== LOAD DATASET ====
 <xarray.Dataset>
Dimensions:                       (time: 14613, latitude: 28, longitude: 28,
                                   level: 13)
Coordinates:
  * latitude                      (latitude) float32 39.0 38.75 ... 32.5 32.25
  * level                         (level) int64 50 100 15

Processing futures: 0it [00:00, ?it/s]

Processing futures: 13it [00:02,  4.46it/s]


4.00889 sec
==== LOAD DATASET ====
 <xarray.Dataset>
Dimensions:                       (time: 14613, longitude: 14, latitude: 33,
                                   level: 13)
Coordinates:
  * latitude                      (latitude) float64 21.0 22.5 ... 67.5 69.0
  * level                         (level) int64 50 100 150 200 ... 850 925 1000
  * longitude                     (longitude) float64 121.5 123.0 ... 141.0
  * time                          (time) datetime64[ns] 2011-12-31 ... 2021-1...
Data variables: (12/15)
    10m_u_component_of_wind       (time, longitude, latitude) float32 ...
    10m_v_component_of_wind       (time, longitude, latitude) float32 ...
    2m_temperature                (time, longitude, latitude) float32 ...
    geopotential                  (time, level, longitude, latitude) float32 ...
    geopotential_at_surface       (longitude, latitude) float32 ...
    mean_sea_level_pressure       (time, longitude, latitude) float32 ...
    ...                     

Processing futures: 13it [00:01,  7.91it/s]


2.29587 sec
==== LOAD DATASET ====
 <xarray.Dataset>
Dimensions:                       (time: 14613, longitude: 17, latitude: 12,
                                   level: 13)
Coordinates:
  * latitude                      (latitude) float64 2.903 8.71 ... 60.97 66.77
  * level                         (level) int64 50 100 150 200 ... 850 925 1000
  * longitude                     (longitude) float64 90.0 95.62 ... 174.4 180.0
  * time                          (time) datetime64[ns] 2011-12-31 ... 2021-1...
Data variables: (12/15)
    10m_u_component_of_wind       (time, longitude, latitude) float32 ...
    10m_v_component_of_wind       (time, longitude, latitude) float32 ...
    2m_temperature                (time, longitude, latitude) float32 ...
    geopotential                  (time, level, longitude, latitude) float32 ...
    geopotential_at_surface       (longitude, latitude) float32 ...
    mean_sea_level_pressure       (time, longitude, latitude) float32 ...
    ...             

Processing futures: 13it [00:00, 16.26it/s]


1.16858 sec
======= RESULT SHAPE =======
result_dataset.shape:  torch.Size([14613, 85, 1450])


In [7]:
print(dataset[0][1].shape)

torch.Size([12, 85, 1450])


In [4]:
import yaml
import logging
import os

config_path = os.path.join(os.path.dirname(os.getcwd()), 'configs/train_config.yaml')

try:
    with open(config_path) as f:
            config_dict = yaml.safe_load(f)
            config: TrainingRunConfig = TrainingRunConfig.parse_obj(config_dict)
except FileNotFoundError:
        logging.error(f"Config file {config_path} does not exist. Exiting.")
except yaml.YAMLError:
        logging.error(f"Config file {config_path} is not valid YAML. Exiting.")

model = VariableAnalyzer(
            var_len=shape[1],
            time_len=time_len,
            dim_model=shape[2],
            predict_dim=dims[0],
            batch_size=config.training.batch_size,
            num_heads=config.model.num_heads,
            n_encoder_layers=config.model.n_encoder_layers,
            n_decoder_layers=config.model.n_decoder_layers,
            dropout=config.model.dropout
        )

checkpoint = torch.load('/workspace/Haea/models/VariableAnalyzer/tb_logs/lightning_logs/bnc1ycog/checkpoints/epoch=19-step=25540.ckpt')

model_pl = TrainModule.load_from_checkpoint('/workspace/Haea/models/VariableAnalyzer/tb_logs/lightning_logs/bnc1ycog/checkpoints/epoch=19-step=25540.ckpt'
                               , model=model, mean_std=mean_std, time_len=time_len, var_len=shape[1], 
                               predict_dim=dims[0], max_iters=config.training.max_epochs*99, 
                               var_lv=dataset_info[0], var_nlv=dataset_info[1], levels=dataset_info[2], config=config.training)

/workspace/Haea/venv/lib/python3.10/site-packages/torch/nn/modules/transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.num_heads is odd
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")
/workspace/Haea/venv/lib/python3.10/site-packages/pytorch_lightning/utilities/parsing.py:198: Attribute 'model' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['model'])`.


In [5]:
device = ("cuda" if torch.cuda.is_available() else "cpu" )
device = torch.device(device)

model = model.to(device)
model.init_seq(device)

init cuda


In [6]:
batch = dataset[1]
src = batch[0].unsqueeze(0).to(device)
tgt = batch[1].unsqueeze(0).to(device)
label = batch[2].unsqueeze(0).to(device)
print(label.shape)
label = label.view(label.size(0), -1, label.size(3))
predict = model(src, tgt)

torch.Size([1, 12, 85, 1450])


In [7]:
print(predict)

tensor([[[ 1.9855e+00,  2.0060e+00,  1.9600e+00,  ...,  2.0277e+00,
           1.9964e+00,  2.0333e+00],
         [ 1.2893e+00,  1.3049e+00,  1.2767e+00,  ...,  1.3063e+00,
           1.3256e+00,  1.3299e+00],
         [ 8.2467e-01,  8.4525e-01,  8.5563e-01,  ...,  8.8770e-01,
           8.9905e-01,  8.7051e-01],
         ...,
         [ 2.4798e-01,  2.4414e-01,  2.4530e-01,  ...,  2.8259e-01,
           2.9866e-01,  2.8730e-01],
         [ 3.1795e-01,  3.0001e-01,  3.0926e-01,  ...,  3.6898e-01,
           3.7406e-01,  3.7367e-01],
         [ 2.4143e-01,  2.3844e-01,  2.0994e-01,  ...,  2.6419e-01,
           2.8906e-01,  2.8419e-01]],

        [[ 2.0073e+00,  2.0086e+00,  1.9760e+00,  ...,  2.0124e+00,
           2.0122e+00,  2.0197e+00],
         [ 1.2939e+00,  1.2682e+00,  1.2514e+00,  ...,  1.3120e+00,
           1.3257e+00,  1.3064e+00],
         [ 8.6862e-01,  8.4383e-01,  8.3227e-01,  ...,  9.0788e-01,
           9.0617e-01,  9.0259e-01],
         ...,
         [ 1.7153e-01,  1